# Rob Hoffman  Detection

In [1]:
import yfinance as yf
#df = yf.download(tickers='BTC-USD', period='2y', interval='1h')
df = yf.download(tickers='EURUSD=X', period='59d', interval='15m')
df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2021-12-30 00:00:00+00:00,1.136622,1.136622,1.135203,1.135203,1.135203,0
2021-12-30 00:15:00+00:00,1.135203,1.135718,1.135203,1.135589,1.135589,0
2021-12-30 00:30:00+00:00,1.135976,1.135976,1.135460,1.135847,1.135847,0
2021-12-30 00:45:00+00:00,1.135589,1.135589,1.135332,1.135460,1.135460,0
2021-12-30 01:00:00+00:00,1.135460,1.135589,1.135203,1.135203,1.135203,0
...,...,...,...,...,...,...
2022-03-22 11:00:00+00:00,1.100837,1.101079,1.100231,1.100594,1.100594,0
2022-03-22 11:15:00+00:00,1.100594,1.101200,1.100110,1.101200,1.101200,0
2022-03-22 11:30:00+00:00,1.100837,1.100837,1.099626,1.100110,1.100110,0


In [2]:
#df=df[df['Volume']!=0]
#df.isna().sum()
df.reset_index(drop=True, inplace=True)
#df.tail()

In [4]:
import pandas_ta as ta
df["EMA"] = ta.ema(df.Close, length=20)

In [5]:
backrollingN = 20
df['slopeEMA'] = df['EMA'].diff(periods=1)
df['slopeEMA'] = df['slopeEMA'].rolling(window=backrollingN).mean()

In [6]:
df.head()

,Open,High,Low,Close,Adj Close,Volume,EMA,slopeEMA
0,1.136622,1.136622,1.135203,1.135203,1.135203,0,NaN,NaN
1,1.135203,1.135718,1.135203,1.135589,1.135589,0,NaN,NaN
2,1.135976,1.135976,1.135460,1.135847,1.135847,0,NaN,NaN
3,1.135589,1.135589,1.135332,1.135460,1.135460,0,NaN,NaN
4,1.135460,1.135589,1.135203,1.135203,1.135203,0,NaN,NaN


## Total signal

In [7]:
TotSignal = [0] * len(df)
slopelimit=5e-5
percentlimit = 0.45
for row in range(0, len(df)):
    if df.slopeEMA[row] < -slopelimit and (min(df.Open[row], df.Close[row])-df.Low[row])/(df.High[row]-df.Low[row])>percentlimit:
        TotSignal[row]=1
    if df.slopeEMA[row] > slopelimit and (df.High[row]-max(df.Open[row], df.Close[row]))/(df.High[row]-df.Low[row])>percentlimit:
        TotSignal[row]=2

df['TotSignal']=TotSignal

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys
D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


In [139]:
df.tail(5)
df.iloc[852]

Open         1.140381
High         1.140901
Low          1.139601
Close        1.139991
Adj Close    1.139991
Volume       0.000000
EMA          1.141416
slopeEMA    -0.000032
TotSignal    0.000000
pointpos          NaN
Name: 852, dtype: float64

## Visualizing signals on chart

In [8]:
import numpy as np
def pointpos(x):
    if x['TotSignal']==1:
        return x['High']+1e-3
    elif x['TotSignal']==2:
        return x['Low']-1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [11]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = df[1800:2200]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=1), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="Signal")
fig.show()